In [1]:
import pandas as pd
import pandas as pd

brain = pd.read_csv("./data/brain.csv")
main = pd.read_csv("./data/data.csv")

brain

,subjectkey,aggressive_sumscore,prosocial_child,prosocial_parent,rsfmri_cor_ngd_cerc_scs_thp,rsfmri_cor_ngd_cerc_scs_cde,rsfmri_cor_ngd_cerc_scs_pt,rsfmri_cor_ngd_cerc_scs_hp,rsfmri_cor_ngd_cerc_scs_ag,rsfmri_cor_ngd_cerc_scs_aa,...,rsfmri_cor_ngd_sa_scs_ag,rsfmri_cor_ngd_sa_scs_aa,rsfmri_cor_ngd_sa_scs_bs,rsfmri_cor_ngd_vta_scs_thp,rsfmri_cor_ngd_vta_scs_cde,rsfmri_cor_ngd_vta_scs_pt,rsfmri_cor_ngd_vta_scs_hp,rsfmri_cor_ngd_vta_scs_ag,rsfmri_cor_ngd_vta_scs_aa,rsfmri_cor_ngd_vta_scs_bs
0,NDAR_INV003RTV85,-1.613056,2.0,5.0,-0.099465,0.011902,0.105324,-0.060815,0.125246,0.137189,...,0.033977,0.176136,0.016017,-0.159200,0.096950,-0.105231,0.093810,-0.146213,-0.065730,0.151937
1,NDAR_INV005V6D2C,-2.020651,6.0,5.0,-0.011021,0.090607,0.035170,0.177055,0.101034,0.290015,...,0.000060,0.030458,0.078567,-0.167591,-0.089897,0.047835,-0.040634,0.013578,0.071460,0.015810
2,NDAR_INV007W6H7B,-2.020651,6.0,6.0,0.021635,0.109315,0.075327,0.023361,0.037646,0.097682,...,0.038526,0.038154,-0.122156,-0.066243,-0.014754,-0.007929,-0.066657,-0.029313,-0.007276,0.091658
3,NDAR_INV00BD7VDC,-1.840037,5.0,4.0,-0.079703,0.045804,0.085608,0.053425,-0.058847,0.149258,...,0.054630,-0.002562,0.071148,-0.062509,0.020820,-0.048073,0.097190,0.075088,0.065463,0.058266
4,NDAR_INV00CY2MDM,9.133184,6.0,4.0,0.080387,0.058346,-0.059340,-0.050470,-0.237829,0.193095,...,-0.208286,-0.034073,0.005343,-0.197360,-0.046674,-0.022899,0.026314,0.014951,0.072129,0.116322
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11275,NDAR_INVZZNX6W2P,1.032541,6.0,3.0,-0.028410,-0.039971,0.067447,-0.048109,0.027526,0.216098,...,-0.009521,0.066135,0.109836,0.096130,0.036480,0.038568,0.248696,0.033755,-0.013448,0.112434
11276,NDAR_INVZZPKBDAC,-2.020651,6.0,6.0,0.039119,0.020714,0.199961,-0.030482,0.266837,0.310542,...,0.130922,0.041752,0.125584,0.126389,-0.009587,-0.116175,0.123111,-0.079003,-0.029040,0.107674
11277,NDAR_INVZZZ2ALR6,-1.659423,5.0,6.0,-0.053251,-0.019495,0.073093,0.001816,0.099020,0.233202,...,-0.051546,0.073093,0.051083,-0.019044,0.026323,-0.086471,0.144194,-0.004044,0.035556,0.186812
11278,NDAR_INVZZZNB0XC,-1.659423,6.0,6.0,0.069969,0.017381,-0.032929,-0.101266,-0.237510,0.448845,...,-0.146889,0.034003,0.004844,0.057078,-0.118433,-0.011791,0.037884,0.066272,0.322386,0.101986


In [2]:
main

,Unnamed: 0,subjectkey,aggressive_sumscore,prosocial_child,prosocial_parent,asr_scr_perstr_t,asr_scr_anxdep_t,asr_scr_withdrawn_t,asr_scr_somatic_t,asr_scr_thought_t,...,parent_rules_q4,parent_rules_q7,su_risk_p_1,su_risk_p_2_3,su_risk_p_4_5,neighborhood1_2_3_p,neighborhood_crime_y,interview_date,interview_age,sex
0,2,NDAR_INV007W6H7B,-2.020651,6.0,6.0,48.0,50.0,50.0,51.0,54.0,...,0,0,4.0,0.0,4.0,4.666667,4.0,02/21/2017,126,M
1,3,NDAR_INV00BD7VDC,-1.840037,5.0,4.0,46.0,52.0,51.0,50.0,54.0,...,0,0,0.0,0.0,0.0,5.000000,3.0,06/12/2018,112,M
2,4,NDAR_INV00CY2MDM,9.133184,6.0,4.0,50.0,51.0,50.0,50.0,50.0,...,0,0,0.0,0.0,0.0,3.666667,5.0,08/22/2017,130,M
3,7,NDAR_INV00LH735Y,-2.020651,6.0,6.0,28.0,50.0,50.0,52.0,50.0,...,2,2,0.0,0.0,0.0,4.000000,5.0,01/29/2018,109,M
4,8,NDAR_INV00LJVZK2,9.773907,5.0,3.0,50.0,50.0,50.0,50.0,50.0,...,0,0,0.0,0.0,0.0,3.000000,4.0,08/19/2017,121,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10173,11843,NDAR_INVZZNX6W2P,1.032541,6.0,3.0,50.0,50.0,53.0,50.0,50.0,...,2,2,3.0,1.0,0.0,5.000000,4.0,03/13/2017,131,M
10174,11844,NDAR_INVZZPKBDAC,-2.020651,6.0,6.0,59.0,52.0,50.0,53.0,50.0,...,0,0,2.0,0.0,0.0,4.000000,5.0,01/20/2018,113,F
10175,11845,NDAR_INVZZZ2ALR6,-1.659423,5.0,6.0,55.0,52.0,50.0,57.0,50.0,...,0,0,0.0,4.0,0.0,4.000000,5.0,06/15/2017,120,F
10176,11846,NDAR_INVZZZNB0XC,-1.659423,6.0,6.0,59.0,54.0,50.0,65.0,55.0,...,0,0,0.0,0.0,0.0,2.000000,4.0,01/03/2017,108,F
